<a href="https://colab.research.google.com/github/revanthmadasu/LifestyleWellbeing/blob/data-preprocessing/LifestyleWellbeing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the data

In [98]:
dataset = pd.read_csv('https://raw.githubusercontent.com/revanthmadasu/LifestyleWellbeing/master/Wellbeing_and_lifestyle_data_Kaggle.csv')
dataset = dataset.drop(10005)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Converting categorial data into numerical

Understanding which objects to convert

In [99]:
object_dataset = dataset.select_dtypes(include=['object']).copy()
object_dataset.head()

,Timestamp,DAILY_STRESS,AGE,GENDER
0,7/7/15,2,36 to 50,Female
1,7/7/15,3,36 to 50,Female
2,7/7/15,3,36 to 50,Female
3,7/7/15,3,51 or more,Female
4,7/7/15,1,51 or more,Female


Converting Timestamp using label encoding. "Finding and replacing" method would take forever. "One-hot encoder" method would creat a lot more unnecessary columns

In [100]:
dataset['Timestamp'] = dataset['Timestamp'].astype('category')
dataset['Timestamp'] = dataset['Timestamp'].cat.codes
dataset.head()

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE,GENDER,WORK_LIFE_BALANCE_SCORE
0,5606,3,2,2,5,0,5,2,0,1,...,7,5,5,1,4,0,5,36 to 50,Female,609.5
1,5606,2,3,4,3,8,10,5,2,2,...,8,2,2,2,3,2,6,36 to 50,Female,655.6
2,5606,2,3,3,4,4,10,3,2,2,...,8,10,2,2,4,8,3,36 to 50,Female,631.6
3,5606,3,3,10,3,10,7,2,5,2,...,5,7,5,1,5,2,0,51 or more,Female,622.7
4,5606,5,1,3,3,10,4,2,4,2,...,7,0,0,2,8,1,5,51 or more,Female,663.9


Using "Finding and replacing" method, since there are values only from 0 to 5, with a arbitrary value of 1/1/2000 in row 10007

In [101]:
dataset['DAILY_STRESS'] = dataset['DAILY_STRESS'].replace(['0','1','2','3','4','5'], [0,1,2,3,4,5])
dataset.head()

,Timestamp,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,...,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE,GENDER,WORK_LIFE_BALANCE_SCORE
0,5606,3,2,2,5,0,5,2,0,1,...,7,5,5,1,4,0,5,36 to 50,Female,609.5
1,5606,2,3,4,3,8,10,5,2,2,...,8,2,2,2,3,2,6,36 to 50,Female,655.6
2,5606,2,3,3,4,4,10,3,2,2,...,8,10,2,2,4,8,3,36 to 50,Female,631.6
3,5606,3,3,10,3,10,7,2,5,2,...,5,7,5,1,5,2,0,51 or more,Female,622.7
4,5606,5,1,3,3,10,4,2,4,2,...,7,0,0,2,8,1,5,51 or more,Female,663.9


In [102]:
dataset['AGE'] = dataset['AGE'].replace(['21 to 35', '36 to 50', '51 or more', 'Less than 20'],[(21+35)/2, (36+50)/2, (51+80)/2, (20/2)])

Using get_dummies method for 'AGE' & 'GENDER' category

In [103]:
dummy_col_df = pd.get_dummies(dataset, columns = ['GENDER'])
dummy_col_df.columns

Index(['Timestamp', 'FRUITS_VEGGIES', 'DAILY_STRESS', 'PLACES_VISITED',
       'CORE_CIRCLE', 'SUPPORTING_OTHERS', 'SOCIAL_NETWORK', 'ACHIEVEMENT',
       'DONATION', 'BMI_RANGE', 'TODO_COMPLETED', 'FLOW', 'DAILY_STEPS',
       'LIVE_VISION', 'SLEEP_HOURS', 'LOST_VACATION', 'DAILY_SHOUTING',
       'SUFFICIENT_INCOME', 'PERSONAL_AWARDS', 'TIME_FOR_PASSION',
       'WEEKLY_MEDITATION', 'AGE', 'WORK_LIFE_BALANCE_SCORE', 'GENDER_Female',
       'GENDER_Male'],
      dtype='object')

Shifting 'WORK_LIFE_BALANCE_SCORE' to the right

In [105]:
dummy_col_df = dummy_col_df[['Timestamp', 'FRUITS_VEGGIES', 'DAILY_STRESS', 'PLACES_VISITED',
       'CORE_CIRCLE', 'SUPPORTING_OTHERS', 'SOCIAL_NETWORK', 'ACHIEVEMENT',
       'DONATION', 'BMI_RANGE', 'TODO_COMPLETED', 'FLOW', 'DAILY_STEPS',
       'LIVE_VISION', 'SLEEP_HOURS', 'LOST_VACATION', 'DAILY_SHOUTING',
       'SUFFICIENT_INCOME', 'PERSONAL_AWARDS', 'TIME_FOR_PASSION',
       'WEEKLY_MEDITATION', 'AGE', 'GENDER_Female',
       'GENDER_Male', 'WORK_LIFE_BALANCE_SCORE']]
X = dummy_col_df.iloc[:, :-1].values
y = dummy_col_df.iloc[:, -1].values

In [75]:
print(X[0,:])

[5607 3 2 2 5 0 5 2 0 1 6 4 5 0 7 5 5 1 4 0 5 0 1 0 0 1 0]


In [76]:
print(X)

[[5607 3 2 ... 0 1 0]
 [5607 2 3 ... 0 1 0]
 [5607 2 3 ... 0 1 0]
 ...
 [3054 4 3 ... 0 0 1]
 [3055 1 1 ... 0 1 0]
 [3056 5 4 ... 0 1 0]]


In [77]:
print(y)

[609.5 655.6 631.6 ... 716.6 682.  651.4]


# Missing Data

In [106]:
missing_data = dummy_col_df.isnull()
missing_data
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("x")

Timestamp
False    15971
Name: Timestamp, dtype: int64
x
FRUITS_VEGGIES
False    15971
Name: FRUITS_VEGGIES, dtype: int64
x
DAILY_STRESS
False    15971
Name: DAILY_STRESS, dtype: int64
x
PLACES_VISITED
False    15971
Name: PLACES_VISITED, dtype: int64
x
CORE_CIRCLE
False    15971
Name: CORE_CIRCLE, dtype: int64
x
SUPPORTING_OTHERS
False    15971
Name: SUPPORTING_OTHERS, dtype: int64
x
SOCIAL_NETWORK
False    15971
Name: SOCIAL_NETWORK, dtype: int64
x
ACHIEVEMENT
False    15971
Name: ACHIEVEMENT, dtype: int64
x
DONATION
False    15971
Name: DONATION, dtype: int64
x
BMI_RANGE
False    15971
Name: BMI_RANGE, dtype: int64
x
TODO_COMPLETED
False    15971
Name: TODO_COMPLETED, dtype: int64
x
FLOW
False    15971
Name: FLOW, dtype: int64
x
DAILY_STEPS
False    15971
Name: DAILY_STEPS, dtype: int64
x
LIVE_VISION
False    15971
Name: LIVE_VISION, dtype: int64
x
SLEEP_HOURS
False    15971
Name: SLEEP_HOURS, dtype: int64
x
LOST_VACATION
False    15971
Name: LOST_VACATION, dtype: int64
x
DAILY_SHOU

# Data Normalization 

In [107]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaled_X = sc.fit_transform(X)
scaled_X[0]

array([ 1.02351069,  0.05359948, -0.57881948, -0.97629415, -0.1789651 ,
       -1.73240727, -0.47757158, -0.72598514, -1.46647256, -0.83468321,
        0.09680395,  0.34172696, -0.24337079, -1.1613115 , -0.03582342,
        0.56925794,  0.77309453, -1.63990996, -0.55391509, -1.21884478,
       -0.40896998,  0.27459299,  0.78746778, -0.78746778])